In [1]:
import pandas as pd

In [3]:
medsl = pd.read_csv('./raw-from-source/precinct_2018.csv', encoding='ISO-8859-1') 

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,2,8,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
az_elections = medsl[medsl['state_fips'] == 4].copy()

In [ ]:
with pd.option_context("display.max_rows", 1000):
    display(az_elections.groupby(['office']).sum())

In [33]:
az_elections[az_elections.office.isin(['Attorney General',
                                      'US Senate',
                                      'Governor',
                                      'Secretary Of State',
                                      'State Treasurer',
                                      'Superintendent Of Public Instruction',
                                      'State Mine Inspector'])]\
    .groupby(['office']).sum()['votes']

office
Attorney General                        2323916
Governor                                2377434
Secretary Of State                      2333759
State Mine Inspector                    2260358
State Treasurer                         2302373
Superintendent Of Public Instruction    2300611
US Senate                               2385263
Name: votes, dtype: int64